***
__SimpleFilter__: filtering imagecollections based on classification bands
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct
import geemask

***
using the geemask.SimpleFilter class directly on a collection
***

In [ ]:
#
# on GEECol_s2scl
#
eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'month')

roidiameter   = 128
eeregion      = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False

eeimagecollection  = geeproduct.GEECol_s2scl().collect(eeregion, eedatefrom, eedatetill, verbose=verbose)
print( geeutils.szimagecollectioninfo(eeimagecollection) )

#
# filter
#
s2sclfilter   = geemask.SimpleFilter('SCL', [8, 9, 10], -95) # 'SCL' band MUST be available in the collection
eeimagecollection = s2sclfilter.filtercollection(eeimagecollection, eeregion, verbose=verbose)
print( geeutils.szimagecollectioninfo(eeimagecollection) )

#
# remark: 
#   properties eeallclscnt, eeselclscnt and eeselclspct are set 
#   in the SimpleFilter.filtercollection call, but should be considered volatile;
#   e.g. GEE_Col.collect(...) will not preserve these
#
for iIdx in range(eeimagecollection.size().getInfo()):
    img = ee.Image(eeimagecollection.toList(eeimagecollection.size()).get(iIdx))
    print(iIdx, geeutils.szISO8601Date(img.date()), 
          f" - all pix: {int(ee.Number(img.get('eeallclscnt')).round().getInfo()):6d}", 
          f" - sel pix: {int(ee.Number(img.get('eeselclscnt')).round().getInfo()):6d}",
          f" - sel pct: {float(ee.Number(img.get('eeselclspct')).multiply(100).round().divide(100).getInfo()):6.2f}%")


In [ ]:
#
# on GEECol_pv333sm
#
eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'month')

roidiameter   = 128 #128/20*333
eeregion      = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False

eeimagecollection  = geeproduct.GEECol_pv333sm().collect(eeregion, eedatefrom, eedatetill, verbose=verbose)
print( geeutils.szimagecollectioninfo(eeimagecollection) )

#
# filter
#
s2sclfilter   = geemask.SimpleFilter('SM', [112, 120, 240, 248], 5) # 'SM' band MUST be available in the collection
eeimagecollection = s2sclfilter.filtercollection(eeimagecollection, eeregion, verbose=verbose)
print( geeutils.szimagecollectioninfo(eeimagecollection) )

#
# remark: 
#   properties eeallclscnt, eeselclscnt and eeselclspct are set 
#   in the SimpleFilter.filtercollection call, but should be considered volatile;
#   e.g. GEE_Col.collect(...) will not preserve these
#
for iIdx in range(eeimagecollection.size().getInfo()):
    img = ee.Image(eeimagecollection.toList(eeimagecollection.size()).get(iIdx))
    print(geeutils.szISO8601Date(img.date()), 
          f" - all pix: {int(ee.Number(img.get('eeallclscnt')).round().getInfo()):6d}", 
          f" - sel pix: {int(ee.Number(img.get('eeselclscnt')).round().getInfo()):6d}",
          f" - sel pct: {float(ee.Number(img.get('eeselclspct')).multiply(100).round().divide(100).getInfo()):6.2f}%")


***
using the geemask.SimpleFilter class with Sentinel 2 GEE_Col products (ndvi, fapar,... products allow filter)
***

In [ ]:
eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'year')

roidiameter   = 128
eeregion      = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False

for threshold in [-100, -95, -50, -20, -10, -5, -2, -1, -0.5, -0.2, -0.1, 0]: # negative thresholds = MAXIMUM allowed
    s2sclfilter        = geemask.SimpleFilter('SCL', [3, 8, 9, 10], threshold)
    eeimagecollection  = geeproduct.GEECol_s2fapar(s2sclfilter).collect(eeregion, eedatefrom, eedatetill, verbose)
    #
    # remark: properties eeallclscnt, eeselclscnt and eeselclspct are NOT available in collection anymore 
    #
    print( f"threshold ({float(threshold):8.2f}) - remaining images: {eeimagecollection.size().getInfo()}" )


***
using the geemask.SimpleFilter class with Prova V GEE_Col products (ndvi... products allow filter)
***

In [ ]:
eepoint       = geeutils.bobspoint
eedatefrom    = geeutils.fleecycloudsday
eedatetill    = eedatefrom.advance(1, 'year')

roidiameter   = 128/20*333
eeregion      = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False

for threshold in [0.001, 1, 2, 5, 10, 20, 50, 80, 90, 95, 100]: # positive thresholds = MINIMUM required (0 is considered negative!)
    pvsmfilter         = geemask.SimpleFilter('SM', [112, 120, 240, 248], threshold)
    eeimagecollection  = geeproduct.GEECol_pv333ndvi(pvsmfilter).collect(eeregion, eedatefrom, eedatetill, verbose)
    #
    # remark: properties eeallclscnt, eeselclscnt and eeselclspct are NOT available in collection anymore 
    #
    print( f"threshold ({float(threshold):8.2f}) - remaining images: {eeimagecollection.size().getInfo()}" )

***
using the geeproduct.S2sclcppfilter class with Sentinel 2 GEE_Col products (ndvi, fapar,... products allow filter)
***

In [ ]:
eepoint       = geeutils.bobspoint
eedatefrom    = ee.Date('2020-01-01')
eedatetill    = eedatefrom.advance(1, 'year')

roidiameter   = 128
eeregion      = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False

eeimagecollection  = geeproduct.GEECol_s2fapar().collect(eeregion, eedatefrom, eedatetill, verbose)
print( geeutils.szimagecollectioninfo(eeimagecollection) )

for threshold in [-100, -95, -90, -80, -70, -60, -50, -40, -30, -20, -10, 0]: # negative thresholds = MAXIMUM allowed
    eeimagecollection  = geeproduct.GEECol_s2fapar(geeproduct.S2sclcppfilter(thresholdpct=threshold)).collect(eeregion, eedatefrom, eedatetill, verbose)
    print( f"threshold ({float(threshold):8.2f}) - remaining images: {eeimagecollection.size().getInfo()}" )

***
using the geeproduct.PV333smfilter class with Proba V GEE_Col products (ndvi,... products allow filter)
***

In [ ]:
eepoint       = geeutils.bobspoint
eedatefrom    = ee.Date('2020-01-01')
eedatetill    = eedatefrom.advance(1, 'year')

roidiameter   = 128
eeregion      = geeutils.squareareaboundsroi(eepoint, roidiameter/2*20)
verbose       = False

eeimagecollection  = geeproduct.GEECol_pv333ndvi().collect(eeregion, eedatefrom, eedatetill, verbose)
print( geeutils.szimagecollectioninfo(eeimagecollection) )

for threshold in [1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]: # positive thresholds = MINIMUM required
    eeimagecollection  = geeproduct.GEECol_pv333ndvi(geeproduct.PV333smfilter(thresholdpct=threshold)).collect(eeregion, eedatefrom, eedatetill, verbose)
    print( f"threshold ({float(threshold):8.2f}) - remaining images: {eeimagecollection.size().getInfo()}" )

#
# estimate range in roi environment 
#
xmin = eeimagecollection.first().clip(eeregion).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xmax = eeimagecollection.first().clip(eeregion).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
#
# show it
#
map = geemap.Map(height='400px')
map.centerObject(eeregion, 13)
map.addLayer(eeimagecollection.first(), {'min':xmin, 'max':xmax, 'opacity':0.9})
map.addLayer(geeutils.outlinegeometryimage(eeregion, 0, 2),       {'palette':'#ff0000'}, 'eeregion')
map
